# puztool.logic_grid: Grid Logic helpers

In [3]:
from puztool import *
from puztool.logic_grid import *

These are tools for working with the sort of logic puzzles where you have a bunch of categories, a bunch of unique labels in each category, and a bunch of declarations about various subsets of these. For example:

-----

Five people (Brita, Galal, Sam, Violet, and Zork) all have different color hair (Blue, Green, Red, Taupe, and Violet) and were born under different astrological signs (Aries, Scorpio, Virgo, Crabbus, or Gahoolie, The Vase of Tulips). Each has a hairspeed of between one and four follicles/second. Additionally, the following are all true:

1. Brita is not an Aries
2. Sam, the Green haired person, and the Virgo are three different people.
3. Violet does not have Green hair.
3. The person with Blue hair is a Scorpio.
4. Brita has Red or Blue hair.
5. Neither Zork nor the Crabbus has Green hair.
6. The Violet-haired person has more letters in their name than the person born under Gahoolie.
6. The Red-haired person has twice the hairspeed of the Crabbus.
7. Violet has Taupe hair or is an Aries, but not both.
8. The total combined hairspeed of all persons is more than 6, but less than 10.

For each person, what color and speed of hair do they have, and what sign were they born under?

-----

We represent a simple grid of values with the `Grid` type, which takes as input a data frame containing all possible values for each category, in no particular order:

In [2]:
categories = dict(
    name = ['Brita', 'Galal', 'Sam', 'Violet', 'Zork'],
    color = ['Blue', 'Green', 'Red', 'Taupe', 'Violet'],
    sign = ['Aries', 'Scorpio', 'Virgo', 'Crabbus', 'Gahoolie']
)
g = Grid(categories)

This `Grid` object has internal grids of booleans indicating which pairs of values are known to be true or false; initially it's all `None`.

In [4]:
g.grids['name']['color']

array([[None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None]], dtype=object)

To set these values, we can use the helper methods `exclude`, `include`, and `requireOne`.

`exclude(*values)` takes any number of values and encodes that they are all mutually exclusive.
`include(*values)` indicates that the specified values all correspond to each other.
`requireOne(value, [options])` indicates that the first value corresponds to exactly one of the chosen options.

In all cases, you don't need to indicate which category your values are in - `Grid` will infer this from the value itself. Thus, we can indicate the most of the rules listed above with:

In [6]:
g.exclude("Brita", "Aries") # Brita and Ares cannot be the same row
g.exclude("Sam", "Green", "Virgo") # No two of these three can be in the same row
g.require("Scorpio", "Blue") # Scorpio and Blue must be in the same row
g.requireOne("Brita", ["Red", "Blue"]) # The row containing Brita must contain Red or Blue
g.exclude("Zork", "Crabbus", "Green") # None of these are in the same row
# Virgo and Aries are the only two signs with the same length, so Galal and Zork must have these
g.requireOne("Galal", ['Virgo', 'Aries'])
g.requireOne("Zork", ['Virgo', 'Aries'])

In an ideal world I'd build a tool for viewing this right here in the notebook, but this isn't that world so instead you can use `g.html_link()` to get a link to the grid-solving tool at http://www.jsingler.de/apps/logikloeser/, prefilled with your grid (use `g.get_link()` if you want plain text instead of a jupyter `HTML` object):

In [8]:
g.html_link()

Following the above link, you can see that the tool there has inferred a bunch of values for you, and you can probably solve the rest yourself using the last two rules. But in a more complicated case, you might want to use an automatic solver, so we'll demo that here, too.


For more complicated setups, you'll want to use the `Solver` class, which is a subclass of `Grid` that exposes [Numberjack](http://numberjack.ucc.ie/doc/) variables for each cell of the logic grids, and also supports additional non-exclusive categories and 

A `Solver` is a wrapper around a `Grid` that has a bunch of [Numberjack](http://numberjack.ucc.ie/doc/) variables for the different values and can impose constraints on them.

In [7]:
s = Solver(g)

In [8]:
print(s.vgrids['Name']['Color'])

[[Name_Color.0.0 in {0,1}, Name_Color.0.1 in {0,1}, Name_Color.0.2 in {0,1}, Name_Color.0.3 in {0,1}],
 [Name_Color.1.0 in {0,1}, Name_Color.1.1 in {0,1}, Name_Color.1.2 in {0,1}, Name_Color.1.3 in {0,1}],
 [Name_Color.2.0 in {0,1}, Name_Color.2.1 in {0,1}, Name_Color.2.2 in {0,1}, Name_Color.2.3 in {0,1}],
 [Name_Color.3.0 in {0,1}, Name_Color.3.1 in {0,1}, Name_Color.3.2 in {0,1}, Name_Color.3.3 in {0,1}]]


When we ask the solver to solve, it'll automatically generate constraints for all of the booleans in the grid, as well as the various sanity constraints (e.g., each name has exactly one color, if a name and a color match then their signs should also match, etc.). But we can also add new constraints on the Numberjack variables, which are accessed by `s.vars`:

In [9]:
s.add(s.vars['Sam','Taupe'] != s.vars['Sam','Aries']) # Sam is Taupe-haired or an Aries, but not both

Having added this, we can now solve `s` using Numberjack; once we've done so, `s.soln_grid` will be a `Grid` with values set according to the solution, and `s.soln` will be a dataframe of the actual solution:

In [10]:
s.solve()

In [13]:
s.soln_grid.html_link()

In [12]:
s.soln

,Name,Color,Sign
0,Brita,Blue,Scorpio
1,Galal,Green,Aries
2,Sam,Taupe,Crabbus
3,Zork,Red,Virgo


## Future Work

Ideally I'd like to expand this to handle grids where the uniqueness constraint is relaxed, and grids where rows have numeric values instead of just choices from a set. For example, imagine if the above puzzle also included that each person had 0, 1, or 2 apples, and specified rules like "Sam had more apples than the Virgo" - you'd want to add another entry to the grid where each category can have a variable set to 0, 1, or 2, but *without* the constraint that each value appears exactly once, and you'd want to be able to specify constraints using syntax something like `s.vars['Sam','Apples'] > s.vars['Taupe','Apples']`. I don't think this is *hard*, per se, but it is *complicated*, and it's unlikely I'll ever get around to it.

It'd also be cool to warn you when there are multiple solutions. Maybe someday.